## pyKalman

In [2]:
import numpy as np
import pylab as pl
from pykalman import KalmanFilter
from merge_events import merge_event_dataframes

from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.functions import col
from pyspark.sql.functions import abs, sqrt, udf
from pyspark.ml.regression import LinearRegression


__authors__ = ['Matt Strautmann']

ModuleNotFoundError: No module named 'pykalman'

In [ ]:
spark = SparkSession.builder.appName("pyK").getOrCreate()
events_list = ['000001000','000001001']    
event_df = merge_event_dataframes(spark, events_list)

In [ ]:

# specify parameters
random_state = np.random.RandomState(0)
transition_matrix = [[1, 0.1], [0, 1]]
transition_offset = [-0.1, 0.1]
observation_matrix = np.eye(2) + random_state.randn(2, 2) * 0.1
observation_offset = [1.0, -1.0]
transition_covariance = np.eye(2)
observation_covariance = np.eye(2) + random_state.randn(2, 2) * 0.1
initial_state_mean = [5, -5]
initial_state_covariance = [[1, 0.1], [-0.1, 1]]

# sample from model
kf = KalmanFilter(
    transition_matrix, observation_matrix, transition_covariance,
    observation_covariance, transition_offset, observation_offset,
    initial_state_mean, initial_state_covariance,
    random_state=random_state
)
xyz, txtytz = kf.sample(
    n_timesteps=50,
    initial_state=initial_state_mean
)

In [ ]:
# estimate state with filtering and smoothing
filtered_state_estimates = kf.filter(observations)[0]
smoothed_state_estimates = kf.smooth(observations)[0]
print(smoothed_state_estimates)

In [ ]:
# draw estimates
pl.figure()
lines_true = pl.plot(states, color='b')
lines_filt = pl.plot(filtered_state_estimates, color='r')
lines_smooth = pl.plot(smoothed_state_estimates, color='g')
pl.legend((lines_true[0], lines_filt[0], lines_smooth[0]),
          ('true', 'filt', 'smooth'),
          loc='lower right'
)
pl.show()